Para poder abordar el análisis deseado, hemos encontrado la siguiente [plataforma](https://www.themoviedb.org/?language=es) de la que extraer los datos:

<br> 

![Logo TMDB](https://www.themoviedb.org/assets/2/v4/logos/v2/blue_long_2-9665a76b1ae401a510ec1e0ca40ddcb3b0cfe45f1d51b77a308fea0845885648.svg)

<br> 

A continuación, y puesto que la obtención de la información necesaria para abordar las hipótesis de nuestro EDA no es inmediata, mostramos la explicación de cómo hacerlo paso a paso, partiendo de la base de que para poder realizar las peticiones a dicha API hemos obtenido nuestra **API KEY** y que, si alguien quisiera replicar el proceso explicado, debería obtener la suya propia [aquí](https://developer.themoviedb.org/v4/reference/intro/getting-started) y almacenarla en la variable que a lo largo del Jupiter Notebook denotamos como *headers_users*.

**Anotación**: A lo largo del documento hemos incluido la explicación para las extracciones de los datos no sólo para ```Netflix``` y ```Amazon Prime Video```, sino también para ```Disney+``` y ```Max```, con el objetivo de utilizarlas en un proyecto futuro que abarque el análisis de más servicios de streaming en España.

In [1]:
#Importamos los paquetes necesarios para la obtención de los datos de nuestro EDA
import requests
import pandas as pd
import numpy as np
import os
import math
from utils import tvseries

# Regiones:

Puesto que nuestro análisis se va a limitar a la geografía de **España**, necesitamos saber cómo filtraremos las películas y las series disponibles en dicha región. Para ello, necesitamos acceder al siguiente endpoint:


![API Regiones](./images/endpoint-regions.png)

In [3]:
url_regions = "https://api.themoviedb.org/3/watch/providers/regions"
res_url_regions = requests.get(url_regions, headers = headers_user)
print(res_url_regions)

<Response [200]>


In [4]:
res_url_regions.json()

{'results': [{'iso_3166_1': 'AD',
   'english_name': 'Andorra',
   'native_name': 'Andorra'},
  {'iso_3166_1': 'AE',
   'english_name': 'United Arab Emirates',
   'native_name': 'United Arab Emirates'},
  {'iso_3166_1': 'AG',
   'english_name': 'Antigua and Barbuda',
   'native_name': 'Antigua & Barbuda'},
  {'iso_3166_1': 'AL', 'english_name': 'Albania', 'native_name': 'Albania'},
  {'iso_3166_1': 'AO', 'english_name': 'Angola', 'native_name': 'Angola'},
  {'iso_3166_1': 'AR',
   'english_name': 'Argentina',
   'native_name': 'Argentina'},
  {'iso_3166_1': 'AT', 'english_name': 'Austria', 'native_name': 'Austria'},
  {'iso_3166_1': 'AU',
   'english_name': 'Australia',
   'native_name': 'Australia'},
  {'iso_3166_1': 'AZ',
   'english_name': 'Azerbaijan',
   'native_name': 'Azerbaijan'},
  {'iso_3166_1': 'BA',
   'english_name': 'Bosnia and Herzegovina',
   'native_name': 'Bosnia & Herzegovina'},
  {'iso_3166_1': 'BB', 'english_name': 'Barbados', 'native_name': 'Barbados'},
  {'iso_31

In [5]:
res_url_regions.json().keys()

dict_keys(['results'])

In [6]:
res_url_regions.json()["results"]

[{'iso_3166_1': 'AD', 'english_name': 'Andorra', 'native_name': 'Andorra'},
 {'iso_3166_1': 'AE',
  'english_name': 'United Arab Emirates',
  'native_name': 'United Arab Emirates'},
 {'iso_3166_1': 'AG',
  'english_name': 'Antigua and Barbuda',
  'native_name': 'Antigua & Barbuda'},
 {'iso_3166_1': 'AL', 'english_name': 'Albania', 'native_name': 'Albania'},
 {'iso_3166_1': 'AO', 'english_name': 'Angola', 'native_name': 'Angola'},
 {'iso_3166_1': 'AR', 'english_name': 'Argentina', 'native_name': 'Argentina'},
 {'iso_3166_1': 'AT', 'english_name': 'Austria', 'native_name': 'Austria'},
 {'iso_3166_1': 'AU', 'english_name': 'Australia', 'native_name': 'Australia'},
 {'iso_3166_1': 'AZ',
  'english_name': 'Azerbaijan',
  'native_name': 'Azerbaijan'},
 {'iso_3166_1': 'BA',
  'english_name': 'Bosnia and Herzegovina',
  'native_name': 'Bosnia & Herzegovina'},
 {'iso_3166_1': 'BB', 'english_name': 'Barbados', 'native_name': 'Barbados'},
 {'iso_3166_1': 'BE', 'english_name': 'Belgium', 'native_n

In [7]:
regions_df = pd.DataFrame(res_url_regions.json()["results"])
regions_df

,iso_3166_1,english_name,native_name
0,AD,Andorra,Andorra
1,AE,United Arab Emirates,United Arab Emirates
2,AG,Antigua and Barbuda,Antigua & Barbuda
3,AL,Albania,Albania
4,AO,Angola,Angola
...,...,...,...
134,XK,Kosovo,Kosovo
135,YE,Yemen,Yemen
136,ZA,South Africa,South Africa
137,ZM,Zambia,Zambia


In [8]:
#Para poder filtrar, necesitamos el código ISO de España
regions_df[regions_df["native_name"] == "Spain"]

,iso_3166_1,english_name,native_name
40,ES,Spain,Spain


# IDs de las plataformas de streaming:

Puesto que nuestro análisis se va a limitar a las plataformas de **Netflix**, **Amazon Prime Video**, **Disney+** y **Max**, necesitamos conocer el identificador de cada una de ellas.

## Series:
Para los identificadores a nivel **serie**, accedemos al siguiente endpoint:


![API Plataformas para Series](./images/endpoint-tvproviders.png)


In [9]:
#Petición a la bbdd para la obtención de las plataformas de streaming existentes para las series en España
url_series_spain_providers= 'https://api.themoviedb.org/3/watch/providers/tv?watch_region=ES'
res_url_series_spain_providers = requests.get(url_series_spain_providers, headers=headers_user)
print(res_url_series_spain_providers)

<Response [200]>


In [10]:
res_url_series_spain_providers.json()

{'results': [{'display_priorities': {'AD': 7,
    'AL': 7,
    'AR': 0,
    'AT': 2,
    'AU': 0,
    'BA': 7,
    'BE': 0,
    'BG': 16,
    'BO': 2,
    'BR': 0,
    'BZ': 2,
    'CA': 1,
    'CH': 2,
    'CL': 0,
    'CO': 0,
    'CR': 2,
    'CZ': 0,
    'DK': 0,
    'DE': 2,
    'DO': 24,
    'EC': 1,
    'EE': 23,
    'ES': 0,
    'FI': 0,
    'EG': 45,
    'FR': 0,
    'GB': 1,
    'GT': 1,
    'GR': 20,
    'HN': 2,
    'HU': 0,
    'HK': 32,
    'HR': 31,
    'IE': 2,
    'IT': 0,
    'IS': 1,
    'JP': 2,
    'JM': 26,
    'KR': 2,
    'LC': 13,
    'LU': 3,
    'LI': 25,
    'LT': 24,
    'LV': 23,
    'ME': 1,
    'MK': 7,
    'MT': 7,
    'MX': 0,
    'NI': 2,
    'NL': 1,
    'NO': 0,
    'NZ': 0,
    'PE': 0,
    'PA': 27,
    'PL': 0,
    'PT': 0,
    'PH': 30,
    'PY': 2,
    'RO': 0,
    'RS': 7,
    'SG': 2,
    'SI': 7,
    'SE': 0,
    'SK': 0,
    'SV': 27,
    'SM': 25,
    'TR': 0,
    'TT': 14,
    'TW': 2,
    'US': 0,
    'VE': 1,
    'UY': 27},
   'display_

In [11]:
res_url_series_spain_providers.json().keys()

dict_keys(['results'])

In [12]:
res_url_series_spain_providers.json()["results"]

[{'display_priorities': {'AD': 7,
   'AL': 7,
   'AR': 0,
   'AT': 2,
   'AU': 0,
   'BA': 7,
   'BE': 0,
   'BG': 16,
   'BO': 2,
   'BR': 0,
   'BZ': 2,
   'CA': 1,
   'CH': 2,
   'CL': 0,
   'CO': 0,
   'CR': 2,
   'CZ': 0,
   'DK': 0,
   'DE': 2,
   'DO': 24,
   'EC': 1,
   'EE': 23,
   'ES': 0,
   'FI': 0,
   'EG': 45,
   'FR': 0,
   'GB': 1,
   'GT': 1,
   'GR': 20,
   'HN': 2,
   'HU': 0,
   'HK': 32,
   'HR': 31,
   'IE': 2,
   'IT': 0,
   'IS': 1,
   'JP': 2,
   'JM': 26,
   'KR': 2,
   'LC': 13,
   'LU': 3,
   'LI': 25,
   'LT': 24,
   'LV': 23,
   'ME': 1,
   'MK': 7,
   'MT': 7,
   'MX': 0,
   'NI': 2,
   'NL': 1,
   'NO': 0,
   'NZ': 0,
   'PE': 0,
   'PA': 27,
   'PL': 0,
   'PT': 0,
   'PH': 30,
   'PY': 2,
   'RO': 0,
   'RS': 7,
   'SG': 2,
   'SI': 7,
   'SE': 0,
   'SK': 0,
   'SV': 27,
   'SM': 25,
   'TR': 0,
   'TT': 14,
   'TW': 2,
   'US': 0,
   'VE': 1,
   'UY': 27},
  'display_priority': 27,
  'logo_path': '/97yvRBw1GzX7fXprcF80er19ot.jpg',
  'provider_name': 

In [13]:
#Creación del df que contiene las plataformas de streaming para las series en España
serie_spain_providers_data = pd.DataFrame(res_url_series_spain_providers.json()["results"])
serie_spain_providers_data

,display_priorities,display_priority,logo_path,provider_name,provider_id
0,"{'AD': 7, 'AL': 7, 'AR': 0, 'AT': 2, 'AU': 0, ...",27,/97yvRBw1GzX7fXprcF80er19ot.jpg,Disney Plus,337
1,"{'AE': 2, 'AG': 11, 'AR': 5, 'AT': 5, 'AZ': 3,...",9,/2E03IAZsX4ZaUqM7tXlctEPMGWS.jpg,Apple TV Plus,350
2,"{'AD': 1, 'AG': 1, 'AL': 1, 'AE': 9, 'AO': 1, ...",1,/dQeAar5H991VYporEjUspolDarG.jpg,Amazon Prime Video,119
3,{'ES': 3},3,/jse4MOi92Jgetym7nbXFZZBI6LK.jpg,Movistar Plus+ €9.99,2241
4,"{'AL': 8, 'AT': 9, 'BA': 8, 'BE': 10, 'CH': 9,...",4,/bZvc9dXrXNly7cA0V4D9pR8yJwm.jpg,Rakuten TV,35
5,{'ES': 5},5,/f6TRLB3H4jDpFEZ0z2KWSSvu1SB.jpg,Movistar Plus+,149
6,"{'CA': 97, 'ES': 7, 'US': 8}",8,/9BgaNQRMDvVlji1JBZi6tcfxpKx.jpg,fuboTV,257
7,"{'AD': 24, 'AL': 26, 'BA': 26, 'BG': 19, 'CZ':...",3,/gQbqEYd0C9uprYxEUqTM589qn8g.jpg,SkyShowtime,1773
8,"{'ES': 9, 'PT': 27}",27,/kO2SWXvDCHAquaUuTJBuZkTBAuU.jpg,Filmin,63
9,"{'AU': 4, 'AT': 67, 'BE': 3, 'CA': 3, 'CH': 52...",3,/jxIBXlxRbCcy7Y4GvOZKszCd0dv.jpg,Hayu,223


In [14]:
serie_spain_providers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   display_priorities  54 non-null     object
 1   display_priority    54 non-null     int64 
 2   logo_path           54 non-null     object
 3   provider_name       54 non-null     object
 4   provider_id         54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [15]:
#El ID de las plataformas es único
serie_spain_providers_data["provider_id"].nunique()

54

In [16]:
#El nombre de las plataformas es único
serie_spain_providers_data["provider_name"].nunique()

54

A continuación, vamos a obtener los identificadores correspondientes a los servicios de transimisión que nos interesan:

In [17]:
#Como no sabemos los nombres exactos para las plataformas, nos aproximamos buscándolos con la función 'contains'
serie_spain_providers_data[serie_spain_providers_data["provider_name"].str.contains("netflix|hbo|max|disney|amazon prime", case=False, regex=True)]

,display_priorities,display_priority,logo_path,provider_name,provider_id
0,"{'AD': 7, 'AL': 7, 'AR': 0, 'AT': 2, 'AU': 0, ...",27,/97yvRBw1GzX7fXprcF80er19ot.jpg,Disney Plus,337
2,"{'AD': 1, 'AG': 1, 'AL': 1, 'AE': 9, 'AO': 1, ...",1,/dQeAar5H991VYporEjUspolDarG.jpg,Amazon Prime Video,119
12,"{'AG': 12, 'AD': 27, 'BA': 29, 'AR': 61, 'BB':...",30,/fksCUZ9QDWZMUwL2LgMtLckROUN.jpg,Max,1899
28,"{'AE': 0, 'AD': 0, 'AG': 0, 'AL': 0, 'AO': 4, ...",5,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,Netflix,8
41,"{'AU': 53, 'BR': 47, 'CA': 111, 'DE': 103, 'ES...",181,/kICQccvOh8AIBMHGkBXJ047xeHN.jpg,Netflix basic with Ads,1796


De entre todos los servicios previos, vamos a quedarnos únicamente con los que nos interesan:

- **Netflix** --> *provider_id = 8*
- **Amazon Prime Video** --> *provider_id = 119*
- **Disney Plus** --> *provider_id = 337*
- **Max** (antes conocido como **HBO Max**) --> *provider_id = 1899*

In [18]:
serie_spain_providers_eda_df = serie_spain_providers_data[serie_spain_providers_data["provider_id"].isin([8, 119, 337, 1899])]
serie_spain_providers_eda_df

,display_priorities,display_priority,logo_path,provider_name,provider_id
0,"{'AD': 7, 'AL': 7, 'AR': 0, 'AT': 2, 'AU': 0, ...",27,/97yvRBw1GzX7fXprcF80er19ot.jpg,Disney Plus,337
2,"{'AD': 1, 'AG': 1, 'AL': 1, 'AE': 9, 'AO': 1, ...",1,/dQeAar5H991VYporEjUspolDarG.jpg,Amazon Prime Video,119
12,"{'AG': 12, 'AD': 27, 'BA': 29, 'AR': 61, 'BB':...",30,/fksCUZ9QDWZMUwL2LgMtLckROUN.jpg,Max,1899
28,"{'AE': 0, 'AD': 0, 'AG': 0, 'AL': 0, 'AO': 4, ...",5,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,Netflix,8


## Películas:
Para los identificadores a nivel **película**, accedemos al siguiente endpoint:


![API Plataformas para Películas](./images/endpoint-movie_providers.png)

In [19]:
#Petición a la bbdd para la obtención de las plataformas de streaming existentes para las películas en España
url_movies_spain_providers = "https://api.themoviedb.org/3/watch/providers/movie?watch_region=ES"
res_url_movies_spain_providers = requests.get(url_movies_spain_providers, headers=headers_user)
print(res_url_movies_spain_providers)

<Response [200]>


In [20]:
res_url_movies_spain_providers.json()

{'results': [{'display_priorities': {'AD': 7,
    'AL': 7,
    'AR': 0,
    'AT': 2,
    'AU': 0,
    'BA': 7,
    'BE': 0,
    'BG': 16,
    'BO': 2,
    'BR': 0,
    'BZ': 2,
    'CA': 1,
    'CH': 2,
    'CL': 0,
    'CO': 0,
    'CR': 2,
    'CZ': 0,
    'DK': 0,
    'DE': 2,
    'DO': 24,
    'EC': 1,
    'EE': 23,
    'ES': 0,
    'FI': 0,
    'EG': 45,
    'FR': 0,
    'GB': 1,
    'GT': 1,
    'GR': 20,
    'HN': 2,
    'HU': 0,
    'HK': 32,
    'HR': 31,
    'IE': 2,
    'IT': 0,
    'IS': 1,
    'JP': 2,
    'JM': 26,
    'KR': 2,
    'LC': 13,
    'LU': 3,
    'LI': 25,
    'LT': 24,
    'LV': 23,
    'ME': 1,
    'MK': 7,
    'MT': 7,
    'MX': 0,
    'NI': 2,
    'NL': 1,
    'NO': 0,
    'NZ': 0,
    'PE': 0,
    'PA': 27,
    'PL': 0,
    'PT': 0,
    'PH': 30,
    'PY': 2,
    'RO': 0,
    'RS': 7,
    'SG': 2,
    'SI': 7,
    'SE': 0,
    'SK': 0,
    'SV': 27,
    'SM': 25,
    'TR': 0,
    'TT': 14,
    'TW': 2,
    'US': 0,
    'VE': 1,
    'UY': 27},
   'display_

In [21]:
res_url_movies_spain_providers.json().keys()

dict_keys(['results'])

In [22]:
res_url_movies_spain_providers.json()["results"]

[{'display_priorities': {'AD': 7,
   'AL': 7,
   'AR': 0,
   'AT': 2,
   'AU': 0,
   'BA': 7,
   'BE': 0,
   'BG': 16,
   'BO': 2,
   'BR': 0,
   'BZ': 2,
   'CA': 1,
   'CH': 2,
   'CL': 0,
   'CO': 0,
   'CR': 2,
   'CZ': 0,
   'DK': 0,
   'DE': 2,
   'DO': 24,
   'EC': 1,
   'EE': 23,
   'ES': 0,
   'FI': 0,
   'EG': 45,
   'FR': 0,
   'GB': 1,
   'GT': 1,
   'GR': 20,
   'HN': 2,
   'HU': 0,
   'HK': 32,
   'HR': 31,
   'IE': 2,
   'IT': 0,
   'IS': 1,
   'JP': 2,
   'JM': 26,
   'KR': 2,
   'LC': 13,
   'LU': 3,
   'LI': 25,
   'LT': 24,
   'LV': 23,
   'ME': 1,
   'MK': 7,
   'MT': 7,
   'MX': 0,
   'NI': 2,
   'NL': 1,
   'NO': 0,
   'NZ': 0,
   'PE': 0,
   'PA': 27,
   'PL': 0,
   'PT': 0,
   'PH': 30,
   'PY': 2,
   'RO': 0,
   'RS': 7,
   'SG': 2,
   'SI': 7,
   'SE': 0,
   'SK': 0,
   'SV': 27,
   'SM': 25,
   'TR': 0,
   'TT': 14,
   'TW': 2,
   'US': 0,
   'VE': 1,
   'UY': 27},
  'display_priority': 27,
  'logo_path': '/97yvRBw1GzX7fXprcF80er19ot.jpg',
  'provider_name': 

In [23]:
#Creación del df que contiene las plataformas de streaming para las películas en España
movies_spain_providers_data = pd.DataFrame(res_url_movies_spain_providers.json()["results"])
movies_spain_providers_data

,display_priorities,display_priority,logo_path,provider_name,provider_id
0,"{'AD': 7, 'AL': 7, 'AR': 0, 'AT': 2, 'AU': 0, ...",27,/97yvRBw1GzX7fXprcF80er19ot.jpg,Disney Plus,337
1,"{'AE': 2, 'AG': 11, 'AR': 5, 'AT': 5, 'AZ': 3,...",9,/2E03IAZsX4ZaUqM7tXlctEPMGWS.jpg,Apple TV Plus,350
2,"{'AD': 1, 'AG': 1, 'AL': 1, 'AE': 9, 'AO': 1, ...",1,/dQeAar5H991VYporEjUspolDarG.jpg,Amazon Prime Video,119
3,{'ES': 3},3,/jse4MOi92Jgetym7nbXFZZBI6LK.jpg,Movistar Plus+ €9.99,2241
4,"{'AL': 8, 'AT': 9, 'BA': 8, 'BE': 10, 'CH': 9,...",4,/bZvc9dXrXNly7cA0V4D9pR8yJwm.jpg,Rakuten TV,35
...,...,...,...,...,...
59,"{'AU': 57, 'CA': 119, 'ES': 94, 'US': 22}",22,/oTQdXIqM9iewlN4MC2nhKB0gHw.jpg,AMC Plus Apple TV Channel,1854
60,"{'AR': 14, 'AU': 67, 'BR': 13, 'CA': 30, 'CL':...",11,/hwsU65QW7A4dbMEWkDpgHyCNcfS.jpg,Kocowa,464
61,"{'AT': 69, 'CH': 54, 'DE': 30, 'FR': 13, 'ES':...",39,/vPZrjHe7wvALuwJEXT2kwYLi0gV.jpg,Arte,234
62,"{'AT': 72, 'BE': 66, 'BR': 92, 'CL': 38, 'CO':...",88,/efu1Cqc63XrPBoreYnf2mn0Nizj.jpg,MGM Plus Amazon Channel,2141


In [24]:
#Hay más plataformas de streaming para películas que para series
print("Número de plataformas de streaming para series:", len(serie_spain_providers_data))
print("Número de plataformas de streaming para películas:", len(movies_spain_providers_data))

Número de plataformas de streaming para series: 54
Número de plataformas de streaming para películas: 64


Vamos a realizar un pequeño análisis análogo al hecho para las series, con el objetivo de obtener los IDs de los servicios correspondientes a las películas:

In [25]:
movies_spain_providers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   display_priorities  64 non-null     object
 1   display_priority    64 non-null     int64 
 2   logo_path           64 non-null     object
 3   provider_name       64 non-null     object
 4   provider_id         64 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.6+ KB


In [26]:
#El ID de las plataformas es único
movies_spain_providers_data["provider_id"].nunique()

64

In [27]:
#El nombre de las plataformas es único
movies_spain_providers_data["provider_name"].nunique()

64

A continuación, vamos a obtener los identificadores correspondientes a los servicios de transimisión que nos interesan:

In [28]:
#Como no sabemos los nombres exactos para las plataformas, nos aproximamos buscándolos con la función 'contains'
movies_spain_providers_data[movies_spain_providers_data["provider_name"].str.contains("netflix|hbo|max|disney|amazon prime", case=False, regex=True)]

,display_priorities,display_priority,logo_path,provider_name,provider_id
0,"{'AD': 7, 'AL': 7, 'AR': 0, 'AT': 2, 'AU': 0, ...",27,/97yvRBw1GzX7fXprcF80er19ot.jpg,Disney Plus,337
2,"{'AD': 1, 'AG': 1, 'AL': 1, 'AE': 9, 'AO': 1, ...",1,/dQeAar5H991VYporEjUspolDarG.jpg,Amazon Prime Video,119
14,"{'AG': 12, 'AD': 27, 'BA': 29, 'AR': 61, 'BB':...",30,/fksCUZ9QDWZMUwL2LgMtLckROUN.jpg,Max,1899
36,"{'AE': 0, 'AD': 0, 'AG': 0, 'AL': 0, 'AO': 4, ...",5,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,Netflix,8
51,"{'AU': 53, 'BR': 47, 'CA': 111, 'DE': 103, 'ES...",181,/kICQccvOh8AIBMHGkBXJ047xeHN.jpg,Netflix basic with Ads,1796


De entre todos los servicios previos, vamos a quedarnos únicamente con los que nos interesan:

- **Netflix** --> *provider_id = 8*
- **Amazon Prime Video** --> *provider_id = 119*
- **Disney Plus** --> *provider_id = 337*
- **Max** --> *provider_id = 1899*

In [29]:
movies_spain_providers_eda_df = movies_spain_providers_data[movies_spain_providers_data["provider_id"].isin([8, 119, 337, 1899])]
movies_spain_providers_eda_df

,display_priorities,display_priority,logo_path,provider_name,provider_id
0,"{'AD': 7, 'AL': 7, 'AR': 0, 'AT': 2, 'AU': 0, ...",27,/97yvRBw1GzX7fXprcF80er19ot.jpg,Disney Plus,337
2,"{'AD': 1, 'AG': 1, 'AL': 1, 'AE': 9, 'AO': 1, ...",1,/dQeAar5H991VYporEjUspolDarG.jpg,Amazon Prime Video,119
14,"{'AG': 12, 'AD': 27, 'BA': 29, 'AR': 61, 'BB':...",30,/fksCUZ9QDWZMUwL2LgMtLckROUN.jpg,Max,1899
36,"{'AE': 0, 'AD': 0, 'AG': 0, 'AL': 0, 'AO': 4, ...",5,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,Netflix,8


## Los identificadores de los servicios de transmisión que vamos a analizar coinciden para películas y series:

 - **Netflix** --> *provider_id = 8*
 - **Amazon Prime Video** --> *provider_id = 119*
 - **Disney+** --> *provider_id = 337*
 - **Max** --> *provider_id = 1899* 

In [30]:
serie_spain_providers_eda_df

,display_priorities,display_priority,logo_path,provider_name,provider_id
0,"{'AD': 7, 'AL': 7, 'AR': 0, 'AT': 2, 'AU': 0, ...",27,/97yvRBw1GzX7fXprcF80er19ot.jpg,Disney Plus,337
2,"{'AD': 1, 'AG': 1, 'AL': 1, 'AE': 9, 'AO': 1, ...",1,/dQeAar5H991VYporEjUspolDarG.jpg,Amazon Prime Video,119
12,"{'AG': 12, 'AD': 27, 'BA': 29, 'AR': 61, 'BB':...",30,/fksCUZ9QDWZMUwL2LgMtLckROUN.jpg,Max,1899
28,"{'AE': 0, 'AD': 0, 'AG': 0, 'AL': 0, 'AO': 4, ...",5,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,Netflix,8


In [31]:
movies_spain_providers_eda_df

,display_priorities,display_priority,logo_path,provider_name,provider_id
0,"{'AD': 7, 'AL': 7, 'AR': 0, 'AT': 2, 'AU': 0, ...",27,/97yvRBw1GzX7fXprcF80er19ot.jpg,Disney Plus,337
2,"{'AD': 1, 'AG': 1, 'AL': 1, 'AE': 9, 'AO': 1, ...",1,/dQeAar5H991VYporEjUspolDarG.jpg,Amazon Prime Video,119
14,"{'AG': 12, 'AD': 27, 'BA': 29, 'AR': 61, 'BB':...",30,/fksCUZ9QDWZMUwL2LgMtLckROUN.jpg,Max,1899
36,"{'AE': 0, 'AD': 0, 'AG': 0, 'AL': 0, 'AO': 4, ...",5,/pbpMk2JmcoNnQwx5JGpXngfoWtp.jpg,Netflix,8


# Películas y series de las plataformas de streaming:

Una vez obtenidos los datos necesarios para realizar el filtrado de películas y series correcto para nuestro estudio, accedemos a los siguientes endpoints:

![API Películas](./images/endpoint-movies.png)
![API Series](./images/endpoint-series.png)

## Netflix:

### Películas:

In [3]:
#Petición a la bbdd para la obtención de las películas de Netflix en España
url_spain_netflix_movies = "https://api.themoviedb.org/3/discover/movie?with_watch_providers=8&watch_region=ES"
res_url_spain_netflix_movies = requests.get(url_spain_netflix_movies, headers=headers_user)
print(res_url_spain_netflix_movies)

<Response [200]>


In [4]:
res_url_spain_netflix_movies.json()

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/3m0j3hCS8kMAaP9El6Vy5Lqnyft.jpg',
   'genre_ids': [878, 53, 18, 27],
   'id': 1125510,
   'original_language': 'es',
   'original_title': 'El hoyo 2',
   'overview': 'After a mysterious leader imposes his law in a brutal system of vertical cells, a new arrival battles against a dubious food distribution method.',
   'popularity': 2554.356,
   'poster_path': '/6lTrlqGTXiVphXW8AmyndcG1pTR.jpg',
   'release_date': '2024-09-27',
   'title': 'The Platform 2',
   'video': False,
   'vote_average': 5.7,
   'vote_count': 536},
  {'adult': False,
   'backdrop_path': '/vGaBqgY8YRzQVUbBMPnd5SmYvL7.jpg',
   'genre_ids': [16, 35],
   'id': 1329336,
   'original_language': 'en',
   'original_title': 'The Bad Guys: Haunted Heist',
   'overview': 'The crafty Bad Guys crew embarks on a high-stakes Halloween heist to swipe a priceless amulet from a spooky mansion. What could go wrong?',
   'popularity': 944.292,
   'poster_path': '/oEJC05Cq

In [5]:
print("Páginas para Netflix España:",res_url_spain_netflix_movies.json()["total_pages"])
print("Resultados para Netflix España:", res_url_spain_netflix_movies.json()["total_results"])

Páginas para Netflix España: 281
Resultados para Netflix España: 5619


In [6]:
netflix_spain_movies = pd.DataFrame(res_url_spain_netflix_movies.json()["results"])
netflix_spain_movies

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/3m0j3hCS8kMAaP9El6Vy5Lqnyft.jpg,"[878, 53, 18, 27]",1125510,es,El hoyo 2,After a mysterious leader imposes his law in a...,2554.356,/6lTrlqGTXiVphXW8AmyndcG1pTR.jpg,2024-09-27,The Platform 2,False,5.700,536
1,False,/vGaBqgY8YRzQVUbBMPnd5SmYvL7.jpg,"[16, 35]",1329336,en,The Bad Guys: Haunted Heist,The crafty Bad Guys crew embarks on a high-sta...,944.292,/oEJC05CqPugMxC4rFu9r6r6vg6m.jpg,2024-10-02,The Bad Guys: Haunted Heist,False,7.400,47
2,False,/eXQ8O0ykuEB8dUJhq2WJll4vXmA.jpg,"[28, 35, 80]",1210794,sv,Strul,"Wrongfully convicted of murder, a clumsy elect...",917.854,/ildbg7ho14EHbt1fZWAkgqIyPqy.jpg,2024-10-02,Trouble,False,7.100,77
3,False,/igtm12Wy9EUlxFeyb4v8bRyuYSY.jpg,"[28, 80, 18, 53]",1337309,th,Bangkok Breaking: ฝ่านรกเมืองเทวดา,When a dedicated rescue worker inadvertently g...,664.257,/6VeDn4oIeUK4XwjWGWMb6qvMImQ.jpg,2024-09-26,Bangkok Breaking: Heaven and Hell,False,6.500,29
4,False,/gZWl93sf8AxavYpVT1Un6EF3oCj.jpg,"[80, 53, 18]",475557,en,Joker,"During the 1980s, a failed stand-up comedian i...",478.179,/udDclJoHjfjb8Ekgsd4FDteOkCU.jpg,2019-10-01,Joker,False,8.151,25441
5,False,/yRKyJJYIzfeiVDHBe4LXguPQCvD.jpg,"[14, 35]",4011,en,Beetlejuice,A newly dead New England couple seeks help fro...,388.911,/nnl6OWkyPpuMm595hmAxNW3rZFn.jpg,1988-03-30,Beetlejuice,False,7.400,7317
6,False,/hdFIdXwS8FSN2wIsuotjW1mshI0.jpg,"[16, 35, 12, 10751]",831815,en,Saving Bikini Bottom: The Sandy Cheeks Movie,"When Bikini Bottom is scooped from the ocean, ...",370.090,/30YnfZdMNIV7noWLdvmcJS0cbnQ.jpg,2024-10-18,Saving Bikini Bottom: The Sandy Cheeks Movie,False,6.700,5
7,False,/3tkDMNfM2YuIAJlvGO6rfIzAnfG.jpg,"[18, 878, 53]",619264,es,El hoyo,A slab of food descends down a vertical facili...,339.031,/8ZX18L5m6rH5viSYpRnTSbb9eXh.jpg,2019-11-08,The Platform,False,6.981,6977
8,False,/jfC2xCBD8a5QJNd5yY3q23E8i7U.jpg,"[28, 80, 18]",1186947,pt,Bandida - A Número Um,"At the age of nine, Rebeca is sold by her gran...",321.382,/rGS8SzheANVQicNba0GCE6w1XHb.jpg,2024-06-20,Outlaw,False,6.000,79
9,False,/nk3RLLZiRAsUxZjtZoXTu77IN03.jpg,"[10751, 35]",1030076,es,Padre no hay más que uno 4: Campanas de boda,"The eldest of the family, Sara, comes of age a...",306.779,/hTMvGolGJLF0GQ7mlwNjbXpsPZ.jpg,2024-07-17,Padre no hay más que uno 4: Campanas de boda,False,6.776,38


Con la anterior petición, sólo hemos sacado la primera página de resultados (por defecto, el endpoint coge la primera página), si bien necesitamos extraer todas las películas (o sea, la información de las 281 páginas correspondientes a Netflix España):

In [7]:
for i in range(2,282): #Empezamos en la página 2 porque la 1 ya la tenemos de la peticion anterior
    url = "https://api.themoviedb.org/3/discover/movie?with_watch_providers=8&watch_region=ES&page="+str(i)
    res_url = requests.get(url,headers=headers_user)
    netflix_spain_movies = pd.concat([netflix_spain_movies, pd.DataFrame(res_url.json()["results"])], ignore_index=True)
netflix_spain_movies

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/3m0j3hCS8kMAaP9El6Vy5Lqnyft.jpg,"[878, 53, 18, 27]",1125510,es,El hoyo 2,After a mysterious leader imposes his law in a...,2554.356,/6lTrlqGTXiVphXW8AmyndcG1pTR.jpg,2024-09-27,The Platform 2,False,5.700,536
1,False,/vGaBqgY8YRzQVUbBMPnd5SmYvL7.jpg,"[16, 35]",1329336,en,The Bad Guys: Haunted Heist,The crafty Bad Guys crew embarks on a high-sta...,944.292,/oEJC05CqPugMxC4rFu9r6r6vg6m.jpg,2024-10-02,The Bad Guys: Haunted Heist,False,7.400,47
2,False,/eXQ8O0ykuEB8dUJhq2WJll4vXmA.jpg,"[28, 35, 80]",1210794,sv,Strul,"Wrongfully convicted of murder, a clumsy elect...",917.854,/ildbg7ho14EHbt1fZWAkgqIyPqy.jpg,2024-10-02,Trouble,False,7.100,77
3,False,/igtm12Wy9EUlxFeyb4v8bRyuYSY.jpg,"[28, 80, 18, 53]",1337309,th,Bangkok Breaking: ฝ่านรกเมืองเทวดา,When a dedicated rescue worker inadvertently g...,664.257,/6VeDn4oIeUK4XwjWGWMb6qvMImQ.jpg,2024-09-26,Bangkok Breaking: Heaven and Hell,False,6.500,29
4,False,/gZWl93sf8AxavYpVT1Un6EF3oCj.jpg,"[80, 53, 18]",475557,en,Joker,"During the 1980s, a failed stand-up comedian i...",478.179,/udDclJoHjfjb8Ekgsd4FDteOkCU.jpg,2019-10-01,Joker,False,8.151,25441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,False,/tJzgYHSGCqNXS2H08n5vBlsuXaV.jpg,[99],999044,nl,Congreso Anyway,,0.001,/cgi4CaJmW0p4K2Y2r7NccR1Qe6b.jpg,1993-04-01,Congreso Anyway,False,0.000,0
5615,False,None,"[99, 36]",920782,fr,Homo sapiens. Une nouvelle histoire de l'homme.,"In this thought-provoking documentary, promine...",0.001,/hS1r8vLqe27j0y042ekpfkoFYVv.jpg,2005-10-29,Homo Futurus: The Inside Story,False,0.000,0
5616,False,/noesJz1tuxcDV75bBjaDkU3fLO.jpg,[],655704,pl,Wild Records!,This cartoon documentary shares trivia and fun...,0.001,/jN3428zsHSxswmQeoPBu8YEVtN8.jpg,2001-02-03,Wild Records!,False,0.000,0
5617,False,None,[],460784,ru,Не все кошки серы,,0.001,/qPL1k2zXtf5qxJJAjHUYDsZOCJS.jpg,2004-01-01,Not all the Cats are Gray,False,0.000,0


Ya tenemos todas las películas de Netlix España. Para nuestro análisis, no necesitamos todos los campos de la tabla creada, por lo que vamos a quedarnos con aquellos que realmente necesitamos:

In [11]:
eda_movies_columns = ['original_title', 'title', 'id', 'genre_ids', 'release_date', 'popularity', 'vote_average', 'vote_count', 'original_language', 'adult' ]

In [12]:
#Generamos la tabla de las películas de Netflix España y la guardamos en un .csv
netflix_spain_movies_eda = netflix_spain_movies[eda_movies_columns]
current_path = os.getcwd()
path_def = os.path.join(current_path, 'data')
netflix_spain_movies_eda.to_csv(os.path.join(path_def,'netflix_movies.csv'))

### Series:

In [39]:
#Petición a la bbdd para la obtención de las series de Netflix en España
url_spain_netflix_series = "https://api.themoviedb.org/3/discover/tv?with_watch_providers=8&watch_region=ES"
res_url_spain_netflix_series = requests.get(url_spain_netflix_series, headers=headers_user)
print(res_url_spain_netflix_series)

<Response [200]>


In [40]:
res_url_spain_netflix_series.json()

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/7w165QdHmJuTHSQwEyJDBDpuDT7.jpg',
   'genre_ids': [10759, 80, 18],
   'id': 2288,
   'origin_country': ['US'],
   'original_language': 'en',
   'original_name': 'Prison Break',
   'overview': 'Due to a political conspiracy, an innocent man is sent to death row and his only hope is his brother, who makes it his mission to deliberately get himself sent to the same prison in order to break the both of them out, from the inside out.',
   'popularity': 985.135,
   'poster_path': '/5E1BhkCgjLBlqx557Z5yzcN0i88.jpg',
   'first_air_date': '2005-08-29',
   'name': 'Prison Break',
   'vote_average': 8.079,
   'vote_count': 5044},
  {'adult': False,
   'backdrop_path': '/tMLH2VDnTpZLYg7T4QPBjaaleVx.jpg',
   'genre_ids': [10759, 18, 9648],
   'id': 100757,
   'origin_country': ['US'],
   'original_language': 'en',
   'original_name': 'Outer Banks',
   'overview': "A tight-knit group of teens unearths a long-buried secret, setting off a

In [41]:
print("Páginas para Netflix España:",res_url_spain_netflix_series.json()["total_pages"])
print("Resultados para Netflix España:", res_url_spain_netflix_series.json()["total_results"])

Páginas para Netflix España: 125
Resultados para Netflix España: 2486


In [42]:
netflix_spain_series = pd.DataFrame(res_url_spain_netflix_series.json()["results"])
netflix_spain_series.head(2)

,adult,backdrop_path,genre_ids,id,origin_country,original_language,original_name,overview,popularity,poster_path,first_air_date,name,vote_average,vote_count
0,False,/7w165QdHmJuTHSQwEyJDBDpuDT7.jpg,"[10759, 80, 18]",2288,[US],en,Prison Break,"Due to a political conspiracy, an innocent man...",985.135,/5E1BhkCgjLBlqx557Z5yzcN0i88.jpg,2005-08-29,Prison Break,8.079,5044
1,False,/tMLH2VDnTpZLYg7T4QPBjaaleVx.jpg,"[10759, 18, 9648]",100757,[US],en,Outer Banks,A tight-knit group of teens unearths a long-bu...,1165.631,/ovDgO2LPfwdVRfvScAqo9aMiIW.jpg,2020-04-15,Outer Banks,8.400,887


Con la anterior petición, sólo hemos sacado la primera página de resultados (por defecto, el endpoint coge la primera página), si bien necesitamos extraer todas las series (o sea, la información de las 125 páginas correspondientes a Netflix España):

In [43]:
for i in range(2,126): #Empezamos en la página 2 porque la 1 ya la tenemos de la peticion anterior
    url = "https://api.themoviedb.org/3/discover/tv?with_watch_providers=8&watch_region=ES&page="+str(i)
    res_url = requests.get(url,headers=headers_user)
    netflix_spain_series = pd.concat([netflix_spain_series, pd.DataFrame(res_url.json()["results"])], ignore_index=True)
netflix_spain_series

,adult,backdrop_path,genre_ids,id,origin_country,original_language,original_name,overview,popularity,poster_path,first_air_date,name,vote_average,vote_count
0,False,/7w165QdHmJuTHSQwEyJDBDpuDT7.jpg,"[10759, 80, 18]",2288,[US],en,Prison Break,"Due to a political conspiracy, an innocent man...",985.135,/5E1BhkCgjLBlqx557Z5yzcN0i88.jpg,2005-08-29,Prison Break,8.079,5044
1,False,/tMLH2VDnTpZLYg7T4QPBjaaleVx.jpg,"[10759, 18, 9648]",100757,[US],en,Outer Banks,A tight-knit group of teens unearths a long-bu...,1165.631,/ovDgO2LPfwdVRfvScAqo9aMiIW.jpg,2020-04-15,Outer Banks,8.400,887
2,False,/4msN2f5WeKVldbdxSymdndWTOmr.jpg,"[16, 10759, 10765]",218843,[JP],ja,七つの大罪 黙示録の四騎士,As a prophecy of doom unfolds on the peaceful ...,534.912,/6SWVJvCIC9B16u6wASZ8FEaNssz.jpg,2023-10-08,The Seven Deadly Sins: Four Knights of the Apo...,8.700,113
3,False,/h324Kf6pjDzGQiUrc4W2r4YcD9L.jpg,"[18, 80]",225634,[US],en,Monsters,"The story of the Menéndez brothers, who were c...",721.391,/x9YC2rpXHUFMqI1hCekKDm9UE4w.jpg,2024-09-19,Monsters,7.586,187
4,False,/cAmpVRgQL2s55CU6K8SaFcUJPNW.jpg,"[18, 35, 10751]",262252,[CO],es,Klass 95: El poder de la belleza,An ambitious young woman launches a powerful m...,388.277,/dsEtoQtIurztGFs3H9vsa5Tdw2R.jpg,2024-08-20,KLASS 95: The Power of Beauty,6.800,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,False,None,[],215052,[US],en,Babies 24/7: The Maternity Ward,The reality of childbirth is laid bare in this...,0.140,None,2022-11-22,Babies 24/7: The Maternity Ward,2.000,1
2482,False,/xX1J1qvhpjPdQs6DBt4GyDTxQeW.jpg,[],121657,[],es,Curso del 63,,0.019,/A06LS444p6u5HUsDadECEHs9sKE.jpg,2009-10-06,Curso del 63,0.000,0
2483,False,None,[],250132,[IT],it,Chi è Jack lo Squartatore,,0.036,None,2020-03-10,Chi è Jack lo Squartatore,0.000,0
2484,False,None,[],249088,[IT],it,Chi era… - Lo show,,0.015,None,2018-07-11,Chi era… - Lo show,0.000,0


Ya tenemos todas las series de Netlix España. Para nuestro análisis, no necesitamos todos los campos de la tabla creada, por lo que vamos a quedarnos con aquellos que realmente necesitamos:

In [44]:
eda_series_columns = ['original_name', 'name', 'id', 'genre_ids', 'first_air_date', 'popularity', 'vote_average', 'vote_count', 'original_language', 'adult' ]

In [45]:
#Generamos la tabla de las series de Netflix España y la guardamos en un .csv
netflix_spain_series_eda = netflix_spain_series[eda_series_columns]
netflix_spain_series_eda.to_csv(os.path.join(path_def,'netflix_series.csv'))

Para el resto de plataformas vamos a aplicar la función ***get_products_tmdb*** definida en ***utils/tvseries.py*** , para obtener sus correspondientes listados de películas y series.

## Amazon Prime Video:

### Películas:

In [46]:
amazon_spain_movies = tvseries.get_products_tmdb("movie", 119, headers_user)
amazon_spain_movies_eda = amazon_spain_movies[eda_movies_columns]
amazon_spain_movies_eda.to_csv(os.path.join(path_def,'amazon_movies.csv'))

### Series:

In [48]:
amazon_spain_series = tvseries.get_products_tmdb("tv", 119, headers_user)
amazon_spain_series_eda = amazon_spain_series[eda_series_columns]
amazon_spain_series_eda.to_csv(os.path.join(path_def,'amazon_series.csv'))

## Disney:

### Películas:

In [50]:
disney_spain_movies = tvseries.get_products_tmdb("movie", 337, headers_user)
disney_spain_movies_eda = disney_spain_movies[eda_movies_columns]
disney_spain_movies_eda.to_csv(os.path.join(path_def,'disney_movies.csv'))

### Series:

In [52]:
disney_spain_series = tvseries.get_products_tmdb("tv", 337, headers_user)
disney_spain_series_eda = disney_spain_series[eda_series_columns]
disney_spain_series_eda.to_csv(os.path.join(path_def,'disney_series.csv'))

## Max:

### Películas:

In [54]:
max_spain_movies = tvseries.get_products_tmdb("movie", 1899, headers_user)
max_spain_movies_eda = max_spain_movies[eda_movies_columns]
max_spain_movies_eda.to_csv(os.path.join(path_def,'max_movies.csv'))

### Series:

In [56]:
max_spain_series = tvseries.get_products_tmdb("tv", 1899, headers_user)
max_spain_series_eda = max_spain_series[eda_series_columns]
max_spain_series_eda.to_csv(os.path.join(path_def,'max_series.csv'))

# Géneros:

En los datasets hasta ahora formados, sólo tenemos los IDs de los géneros, si bien no sabemos cuáles son estos. Así pues, vamos a buscar la información de estos en los siguientes endpoints:

## Películas:

![API Géneros Películas](./images/endpoint-movie_genres.png)

In [74]:
url_movie_genres = "https://api.themoviedb.org/3/genre/movie/list"
res_url_movie_genres = requests.get(url_movie_genres, headers=headers_user)
res_url_movie_genres

<Response [200]>

In [75]:
movie_genres_df = pd.DataFrame(res_url_movie_genres.json()["genres"])
movie_genres_df

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


In [76]:
#Almacenamos la tabla previa para guardarla en .csv
movie_genres_df.to_csv(os.path.join(path_def,'tmdb_movie_genres.csv'))

## Series:


![API Películas](./images/endpoint-serie_genres.png)

In [77]:
url_serie_genres = "https://api.themoviedb.org/3/genre/tv/list"
res_url_serie_genres = requests.get(url_serie_genres, headers=headers_user)
res_url_serie_genres

<Response [200]>

In [78]:
serie_genres_df = pd.DataFrame(res_url_serie_genres.json()["genres"]).reset_index(drop=True)
serie_genres_df

,id,name
0,10759,Action & Adventure
1,16,Animation
2,35,Comedy
3,80,Crime
4,99,Documentary
5,18,Drama
6,10751,Family
7,10762,Kids
8,9648,Mystery
9,10763,News


In [79]:
#Almacenamos la tabla previa para guardarla en .csv
serie_genres_df.to_csv(os.path.join(path_def,'tmdb_series_genres.csv'))